In [ ]:
#!pip install openai
import os
import openai
import pandas as pd
import numpy as np
import time
import random
openai.organization = "organization_id"
openai.api_key = "api_key"

In [ ]:
##### Load data #####
ethos_data = pd.read_csv("Ethos_Dataset_Multi_Label.csv", delimiter=';')
ethos_data_binary = pd.read_csv("Ethos_Dataset_Binary.csv", delimiter=';')
sexist = pd.DataFrame(ethos_data[ethos_data["gender"] >= 0.5]["comment"])
racist = pd.DataFrame(ethos_data[ethos_data["race"] >= 0.5]["comment"])
not_hate = ethos_data_binary[ethos_data_binary["isHate"] == 0]
sexist["category"] = "sexist"
racist["category"] = "racist"
not_hate["category"] = "not hate"

### Other categories ###
#disability = ethos_data[ethos_data["disability"] >= 1]["comment"]
#religion = ethos_data[ethos_data["religion"] >= 1]["comment"]
#sexual_orientation = ethos_data[ethos_data["sexual_orientation"] >= 1]["comment"]

In [ ]:
##### Zero-Shot #####
def zero_shot(cat, label, comment, temperature):
    prompt='Is the following text '+cat+', answer yes or no. '+' "'+comment+'" '+' Answer:'
    zero_shot_results = dict()
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        temperature=temperature,
        max_tokens=2,
        top_p=1,
        n = 1,
        stop=["\n"]
    )
    
    zero_shot_results["category"] = cat
    zero_shot_results["label"] = label
    zero_shot_results["prompt"] = prompt
    zero_shot_results['answer'] = response["choices"][0].text
    zero_shot_results['temperature'] = temperature
    return zero_shot_results

In [ ]:
# One-Shot
def one_shot(cat, label, example, comment, temperature):
    one_shot_results = dict()
    prompt = 'The following text in quotes is ' + cat+ ': ' +\
        '"'+example + '"' +\
        " Is the following text in quotes " + cat+ ", answer yes or no: " +\
        '"'+comment + '."'+\
        " Answer:"
    response = openai.Completion.create(
        engine="davinci",
        prompt = prompt,
        temperature=0,
        max_tokens=2,
        top_p=1,
        n = 1,
        stop=["\n"]
    )
    one_shot_results["category"] = cat
    one_shot_results["label"] = label
    one_shot_results["prompt"] = prompt
    one_shot_results["example"] = example
    one_shot_results["comment"] = comment
    one_shot_results['answer'] = response["choices"][0].text
    one_shot_results['temperature'] = temperature
    
    return one_shot_results

In [ ]:
# Few-Shot: single-category
def few_shot_single(cat, label, example1, example2, example3, comment, temperature):
    few_shot_results = dict()
    prompt = "'''"+example1+"'''"+ ": " + cat+'. \n'+\
        "'''"+example2+"'''"+ ": " + "not-"+ cat +'. \n'+\
        "'''"+example3+"'''"+ ": " + cat+'. \n'+\
        "'''"+comment+"''':"
    response = openai.Completion.create(
        engine="davinci",
        prompt= prompt,
        temperature=0,
        max_tokens=4,
        top_p=1,
        n = 1,
        stop=["."]
    )
    few_shot_results["category"] = cat
    few_shot_results["label"] = label
    few_shot_results["prompt"] = prompt
    few_shot_results["example1"] = example1
    few_shot_results["example2"] = example2
    few_shot_results["example3"] = example3
    few_shot_results["comment"] = comment
    few_shot_results['answer'] = response["choices"][0].text
    few_shot_results["temperature"] = temperature
    
    return few_shot_results

In [ ]:
# Few-Shot: mixed-category
def few_shot_mixed(cat, label, example1, example2, example3, comment, temperature):
    few_shot_results = dict()
    example1 = sexist.reset_index()["comment"][i]
    example2 = racist.reset_index()["comment"][i]
    example3 = not_hate.reset_index()["comment"][i]
    instruction = "Classify the following texts into 'racist', 'sexist' or 'neither'. \n"
    prompt = instruction +\ # Remove instruction to not include an instruction in the prompt
        "'''"+example1+"'''"+ ": " + "sexist. \n"+\
        "'''"+example2+"'''"+ ": " + "racist. \n"+\
        "'''"+example3+"'''"+ ": " + "neither. \n"+\
        "'''"+comment+"''':"
    response = openai.Completion.create(
        engine="davinci",
        prompt= prompt,
        temperature=0,
        max_tokens=4,
        top_p=1,
        n = 1,
        stop=["."]
    )
    few_shot_results["category"] = cat
    few_shot_results["label"] = label
    few_shot_results["prompt"] = prompt
    few_shot_results["example1"] = example1
    few_shot_results["example2"] = example2
    few_shot_results["example3"] = example3
    few_shot_results["example_set"] = i
    few_shot_results["comment"] = comment
    few_shot_results['answer'] = response["choices"][0].text
    few_shot_results["temperature"] = temperature
    
    return few_shot_results

In [ ]:
def few_shot_instruction(cat, label, example1, example2, example3, comment, temperature):
    few_shot_results = dict()
    prompt = "Classify the following texts into 'racist', 'sexist' or 'neither'. \n" +\
        "'''"+example1+"'''"+ ": " + "sexist. \n"+\
        "'''"+example2+"'''"+ ": " + "racist. \n"+\
        "'''"+example3+"'''"+ ": " + "neither. \n"+\
        "'''"+comment+"''':"
    response = openai.Completion.create(
        engine="davinci",
        prompt= prompt,
        temperature=0,
        max_tokens=4,
        top_p=1,
        n = 1,
        stop=["."]
    )
    few_shot_results["category"] = cat
    few_shot_results["label"] = label
    few_shot_results["prompt"] = prompt
    few_shot_results["example1"] = example1
    few_shot_results["example2"] = example2
    few_shot_results["example3"] = example3
    few_shot_results["comment"] = comment
    few_shot_results['answer'] = response["choices"][0].text
    few_shot_results["temperature"] = temperature
    
    return few_shot_results